<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Using Python package

In this notebook, we will show you how to use a more advanced python package called SciPy to solve the MDA and MDO problem using the same models, and the same implementation as in [notebook n°2](02_pure_python.ipynb). The models are described in detail in said notebook and they won't be re-explained here. The reader is encouraged to open the notebook in case there is a question on the models.

## Solving the MDA 

In order to solve the MDA, we will used the `fsolve` function from the `scipy.optimize` package (documentation is available [here](https://docs.scipy.org/doc/scipy/index.html)). The `fsolve` function will solve the equation f(x)=0 so we will need to change slightly the implementation. We will define the function that computes, for a given **MTOW**, the difference between the updated **MTOW** and the actual **MTOW**, which is similar to how we solve the dichotomy problem from the first notebook, and feed this function to the `fsolve` function.

Let's start by redefining the constant from the MDA that we had in the previous notebook.

In [ ]:
# Let's start by defining the characteristics related to ...

# ... the geometry
wing_loading = 115.0  # in kg/m2
aspect_ratio = 10.0  # no unit

# ... the target mission
cruise_altitude = 2500.0  # in m
cruise_speed = 80.0  # in m/s
mission_range = 1389000.0  # in m, 750 nm
payload = 320.0  # in kg, corresponds to 4 passengers of 80 kg

# ... the propulsion technology
tsfc = 7.3e-6 # in kg/N/s

Now, let's define the function to solve, as previously said, we will define it so that when this function equals to zero, we will have a converged aircraft.

In [ ]:
from modules.pure_python.mtow_loop import mtow_loop


def MTOW_to_solve(
    mtow, aspect_ratio, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
):

    MTOW_diff = (
        mtow_loop(
            mtow,
            aspect_ratio,
            wing_loading,
            cruise_altitude,
            cruise_speed,
            mission_range,
            payload,
            tsfc,
        )
        - mtow
    )

    return MTOW_diff

The reader will notice that even if we only want to solve for the **MTOW**, we still have to pass every argument that the `mtow_loop` functions needs even if there are constant. Luckily `fsolve` allows to pass arguments that do not vary for the resolution of the problem. Let's then find the **MTOW** that gives us a converged aircraft !

In [ ]:
from scipy.optimize import fsolve
import numpy as np

# Let's use fsolve in the most simple way possible
initial_guess = 500.0

MTOW_result = fsolve(
    MTOW_to_solve,  # Giving the name of the function we want to solve
    initial_guess,  # The initial guess for the solution
    args=(
        aspect_ratio,
        wing_loading,
        cruise_altitude,
        cruise_speed,
        mission_range,
        payload,
        tsfc,
    ),  # The arguments that won't vary during the problem
    xtol=0.01,  # The relative error between two iterations of the problem
)

print("Result for the MTOW :", float(np.round(MTOW_result, 1)), "kg")

The result is very close to the first one even though the tolerance aren't. Indeed, the first method used absolute difference, here because of `fsolve`, it is a relative error. 

Let's now move on to the MDO problem.

## MDO solving

As in the first notebook, we will here try to find the aspect ratio that gives the least fuel consumed on the design mission. To do that we will first use the same simple method as in the first notebook (meaning exploring a range of aspect ratio, solving the problem, this time with fsolve, and then see which **AR** gives the lowest fuel consumed), and then use another functionnality of `scipy.optimize` called `minimize`.

Let's start with the simple method. For the computation of the fuel consumption and to avoid having to write a new function, we will reuse the `mtow_loop_and_fuel` and only keep the second returned variable which correspond to the fuel consumed. We must however be sure to use a converged **MTOW** as its input for the result to be realistic.

In [ ]:
from modules.pure_python.mtow_loop_and_fuel import mtow_loop_and_fuel
import matplotlib.pyplot as plt

AR_array = np.linspace(1.0, 20.0, 100)
fuel_result = np.zeros_like(AR_array)

for aspect_ratio in AR_array:
    mtow = fsolve(
        MTOW_to_solve,  # Giving the name of the function we want to solve
        initial_guess,  # The initial guess for the solution
        args=(
            aspect_ratio,
            wing_loading,
            cruise_altitude,
            cruise_speed,
            mission_range,
            payload,
            tsfc,
        ),  # The arguments that won't vary during the problem
        xtol=0.01,  # The relative error between two iterations of the problem
    )
    
    # We are here using the function from the previous notebook that jointly computes 
    # the MTOW and the fuel consumption. Since the MTOW has already converged, we will
    # only extract the fuel which corresponds to the fuel on the sizing mission
    fuel_result[np.where(AR_array == aspect_ratio)[0]] = mtow_loop_and_fuel(
        mtow,
        aspect_ratio,
        wing_loading,
        cruise_altitude,
        cruise_speed,
        mission_range,
        payload,
        tsfc,
    )[1]


plt.plot(AR_array, fuel_result)
plt.show()

min_fuel_AR = AR_array[np.where(fuel_result == np.min(fuel_result))[0]]
print("The aspect ratio which gives the minimum MTOW is ", np.round(min_fuel_AR[0], 1))
print("The corresponding fuel consumption is ", np.round(np.min(fuel_result), 1), "kg")

As for the MDA, we have the same results as in the [previous notebook](02_pure_python.ipynb), which is a good sign.

Let's now move on to solving this problem using the `minimize` function from `scipy.optimize`. This function can find the minimum of the function given in parameter. Since we want to minimize the fuel consumption based on the aspect ratio we will give it a function that outputs the fuel consumed on the mission and which has the aspect ratio as one of the inputs. 

The issue however, is that as the aspect ratio varies so will the **MTOW**, so we will also need to resize the aircraft for each **AR**. For the sake of this exercise this can be seen as a constraint to our optimization. So, as an added constraint to our problem, we will tell the `minimize` function that it can modify the aspect ratio but only under the condition that `MTOW_to_solve` is equal to zero, meaning we have a converged aircraft.

All in all, there will be two design variables for this optimization which are going to be the **MTOW** and the **AR** so that we can find the optimum fuel consumption while ensuring we have a sized aircraft.

Here, a couple problem arises from the formalism imposed by the `minimize` function:
* The minimize function only works for objective functions that return a single scalar output which means that we won't be able to reuse `mtow_loop_and_fuel`
* Since we want to ensure that the MTOW is converged, there will be two functions of interest that we provide the `minimize` function with. This imposes that both function have to have the same inputs.
* `minimize` can only take a single object as optimization variable. It can however be a tuple or an array when the function takes in reality more than one variable. Their Python implementation though will need to use only a single variable if it is to be used with `minimize`. Consequently, we will need to reformat our functions so that it can take a tuple as input - tuple that will contain the **MTOW** in first position and the aspect ratio in second - and optimize the function using this tuple.

The implementation of the function that returns the fuel consumed can go two ways. Either we reuse the `mtow_loop_and_fuel` that we modify to accept the right entry, or, create a function that runs the performance module with all the other module needed beforehand (geometry for wing area, aerodynamics for lift-to-drag-ratio and weight for **OWE**). The latter solution was chosen and was implemented in a function called `compute_fuel_scipy` located in the [compute_fuel_scipy.py file](modules/python_for_scipy/compute_fuel_scipy.py).

The reformatting of the `mtow_loop` implementation is done in the [mtow_loop_scipy.py file](modules/python_for_scipy/mtow_loop_scipy.py). As with the original `mtow_loop` function we will then wrap it in a function that returns 0 when the aircraft is converged. This is done in the cell below.

In [ ]:
from modules.python_for_scipy.mtow_loop_scipy import mtow_loop_scipy

def MTOW_to_solve_scipy(
    x, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
):

    MTOW_diff = (
        mtow_loop_scipy(
            x,
            wing_loading,
            cruise_altitude,
            cruise_speed,
            mission_range,
            payload,
            tsfc,
        )
        - x[0]
    )

    return MTOW_diff

We won't be able to use this function to solve the simple MDA problem previously defined. Indeed, since our variable now consists of the **MTOW** and the aspect ratio, we cannot treat the aspect ratio as an argument and thus fix this value. 

Before moving on to the minimization of the fuel consumption, let's do a sanity check and ensure that the solution we found earlier for the MDA is still true (as it should).

In [ ]:
# The print should return 0
x = (1065.8, 10.0)
result = MTOW_to_solve_scipy(
    x, wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc
)
print(
    "The results of a simple evaluation of the MTOW_to_solve_scipy is ", np.round(result, 1), "kg"
)

Now that we've seen that the function still gives the expected results. Let's now redefine our optimization using the new implementation of the method and see what the results are.

In [ ]:
from scipy.optimize import minimize
from modules.python_for_scipy.compute_fuel_scipy import compute_fuel_scipy

# Let's start by defining an initial guess of the couple (MTOW, aspect_ratio), tough we now the solution, and
# could thus accelrate the process, we will start at a different value to test the function.
x0 = (500.0, 5.0)

# We can now define the bounds of the optimization. Essentially there are two constraints : both value must
# be positive, but to avoid looking for unfeasible value, we will take the same bounds as the arrays
# previously used for solving the problems
bnds = ((500.0, 5000.0), (1.0, 20.0))

# We can also define the argument of the function
arguments = (wing_loading, cruise_altitude, cruise_speed, mission_range, payload, tsfc)

# Let's now define the constraints, MTOW_to_solve is equal to 0
cons = {"type": "eq", "fun": MTOW_to_solve_scipy, "args": arguments}

# We can now finally call the minimizer
res = minimize(
    compute_fuel_scipy, x0, method="SLSQP", bounds=bnds, constraints=cons, args=arguments
)

print("The optimized fuel consumption is equal to ", np.round(res.fun, 1), "kg")
print("The corresponding aspect ratio is :", np.round(res.x[1], 1))
print("The corresponding MTOW is :", np.round(res.x[0], 1), "kg")

The results we get are very close to the one we obtained with our optimization "by hand" both in term of the value of the optimization objective and in term of value of the design variable. Except that this time, we can get a more precise result, one not limited by the space between two elements in AR_array. We also managed to find a more precise solution, with less iteration. Only 25 were necessary here whereas we had 100 elements in AR_array, which is a considerable step up.

However the conclusion is pretty much the same as in the first notebook, we have to be careful of unit conversions. Besides data management will only get harder with more modules and the structure of the code is very rigid. For instance, we cannot easily find the optimum (aspect ratio, cruise speed) couple as in order to use the minimize function, the syntax of the function to `minimize` or in the constraints would need to be changed again be so that the variable to optimize comes first in an array and then the arguments (data that don't change value). This would force us to rewrite some of our functions.